In [17]:
import requests, json, io
from PIL import Image

In [129]:
wop = json.load(open(r'C:\Users\Nika\Downloads\default-cards-20240715090650.json', 'rb'))
print(len(wop), 'entries')

96589 entries


In [152]:
layouts = set()
objects = set()

def add(item, database):
    try:
        assert 'paper' in item['games']
        assert item['layout'] not in [
            'art_series', 
            'double_faced_token', 
            'token', 
            'planar',
            'scheme',
            'token',
            'vanguard',
            'emblem'
        ]
        layouts.add(item['layout'])
        objects.add(item['object'])
        if item['oracle_id'] not in database:
            database[item['oracle_id']] = {
                'oracle_id' : item['oracle_id'],
                'name' : item['name'],
                'variants' : [],
                'type_line' : item['type_line'],
                'oracle_text' : item['oracle_text'],
                'mana_cost' : item['mana_cost'],
                'cmc' : item['cmc'],
                'colors' : item['colors'],
                'color_id' : item['color_identity'],
                'keywords' : item['keywords'],
                'legalities' : item['legalities']
            }
    #     else:
    #         if item['name'] != database[item['oracle_id']]['name']:
    #             print(item['name'], database[item['oracle_id']]['name'])
        database[item['oracle_id']]['variants'] += [{
            'set_name' : item['set_name'],
            'set_code' : item['set'],
            'collector_number' : item['collector_number'],
            'finishes' : item['finishes'],
            'artist' : item['artist'],
            'thumb' : item['image_uris']['small'],
            'image' : item['image_uris']['normal'],
            'language' : item['lang'],
            'released' : item['released_at'],
            'rarity' : item['rarity']
        }]
    except:
        pass

oracleDatabase = {}
for i in wop:
    if 'oracle_id' in i:
        add(i, oracleDatabase)
    else:
        continue
#         i['oracle_id'] = i['card_faces'][0]['oracle_id']
#         add(i, oracleDatabase)
#         oracleDatabase[i['oracle_id']]['flip'] = i['card_faces'][1]['oracle_id']
        

In [153]:
layouts

{'adventure',
 'augment',
 'case',
 'class',
 'flip',
 'host',
 'leveler',
 'meld',
 'modal_dfc',
 'mutate',
 'normal',
 'prototype',
 'saga',
 'split',
 'transform'}

In [154]:
print(len(oracleDatabase), 'entries')

keys = oracleDatabase.keys()
dbsize = 0
for key in keys:
    dbsize += len(json.dumps(oracleDatabase[key]))
    
print('{:.2f} mb'.format(dbsize / 1000 / 1000))

27962 entries
59.87 mb


In [151]:
[oracleDatabase[i] for i in oracleDatabase if 'Wojek Investigator' in oracleDatabase[i]['name']]

[{'oracle_id': 'ab41243d-b178-4ebe-a7ac-bea37427be99',
  'name': 'Wojek Investigator',
  'variants': [{'set_name': 'Murders at Karlov Manor Promos',
    'set_code': 'pmkm',
    'collector_number': '36p',
    'finishes': ['nonfoil', 'foil'],
    'artist': 'Ben Hill',
    'thumb': 'https://cards.scryfall.io/small/front/0/9/091748a0-6a01-40cb-9ab8-015aa8235d3c.jpg?1707510568',
    'image': 'https://cards.scryfall.io/normal/front/0/9/091748a0-6a01-40cb-9ab8-015aa8235d3c.jpg?1707510568',
    'language': 'en',
    'released': '2024-02-09',
    'rarity': 'rare'},
   {'set_name': 'Murders at Karlov Manor',
    'set_code': 'mkm',
    'collector_number': '36',
    'finishes': ['nonfoil', 'foil'],
    'artist': 'Ben Hill',
    'thumb': 'https://cards.scryfall.io/small/front/2/9/296574c6-3933-4ab3-b591-72514b244da9.jpg?1706241546',
    'image': 'https://cards.scryfall.io/normal/front/2/9/296574c6-3933-4ab3-b591-72514b244da9.jpg?1706241546',
    'language': 'en',
    'released': '2024-02-09',
    '

In [37]:
scryfallUri = "https://api.scryfall.com/"

def Search(searchValue):
    searchUri = ""
    for char in searchValue.lower():
        if char in "abcdefghijklmnopqrstuvwxyz0123456789 ":
            searchUri += char
    searchUri = searchUri.replace(' ', '+')
    resp = requests.get(scryfallUri + "cards/named?fuzzy=" + searchUri)
    return json.loads(resp.text)

def Variants(cardObject):
    oracleId = cardObject['oracle_id']
    resp = requests.get(scryfallUri + "cards/search?order=released&unique=prints&q=game%3Apaper+oracleid%3A" + oracleId)
    vdat = json.loads(resp.text)['data']
    vlst = []
    for i in vdat:
        cdat = {
            'finishes' : i['finishes'],
            'scryfall' : i['id'],
            'thumb' : i['image_uris']['small'],
            'image' : i['image_uris']['normal']
        }
        vlst += [["{} ({}) {}".format(i['set_name'], i['set'], i['collector_number']), cdat]]
    return vlst

In [53]:
wok = Variants(Search('wojek invest'))
wok

[['Murders at Karlov Manor Promos (pmkm) 36s',
  {'finishes': ['foil'],
   'scryfall': '97b2df60-f773-4b61-9101-05642a227cf5',
   'thumb': 'https://cards.scryfall.io/small/front/9/7/97b2df60-f773-4b61-9101-05642a227cf5.jpg?1707551491',
   'image': 'https://cards.scryfall.io/normal/front/9/7/97b2df60-f773-4b61-9101-05642a227cf5.jpg?1707551491'}],
 ['Murders at Karlov Manor Promos (pmkm) 36p',
  {'finishes': ['nonfoil', 'foil'],
   'scryfall': '091748a0-6a01-40cb-9ab8-015aa8235d3c',
   'thumb': 'https://cards.scryfall.io/small/front/0/9/091748a0-6a01-40cb-9ab8-015aa8235d3c.jpg?1707510568',
   'image': 'https://cards.scryfall.io/normal/front/0/9/091748a0-6a01-40cb-9ab8-015aa8235d3c.jpg?1707510568'}],
 ['Murders at Karlov Manor (mkm) 429',
  {'finishes': ['foil'],
   'scryfall': '58eccc85-bdfe-48f4-a79e-76d9ef122815',
   'thumb': 'https://cards.scryfall.io/small/front/5/8/58eccc85-bdfe-48f4-a79e-76d9ef122815.jpg?1712996935',
   'image': 'https://cards.scryfall.io/normal/front/5/8/58eccc85-

In [51]:
[Image.open(io.BytesIO(requests.get(i[1]['thumb']).content)) for i in wok]

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=146x204>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=146x204>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=146x204>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=146x204>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=146x204>]

In [38]:
offlineKeys = {
    'object' : True,
    'id': True,
    'oracle_id' : True,
    'multiverse_ids' : False,
    'mtgo_id' : False,
    'mtgo_foil_id' : False,
    'tcgplayer_id' : False,
    'cardmarket_id' : False,
    'name' : True,
    'lang' : True,
    'released_at' : True,
    'uri' : False,
    'scryfall_uri' : False,
    'layout' : True,
    'highres_image' : False,
    'image_status' : False,
    'image_uris' : False,
    'mana_cost' : True,
    'cmc' : True,
    'type_line' : True,
    'oracle_text' : True,
    'power' : True,
    'toughness' : True,
    'colors' : True,
    'color_identity' : True,
    'keywords' : True,
    'all_parts' : False,
    'legalities' : True,
    'games' : False,
    'reserved' : False,
    'foil' : False,
    'nonfoil' : False,
    'finishes' : False,
    'oversized' : False,
    'promo' : False,
    'reprint' : False,
    'variation' : False,
    'set_id' : False,
    'set_name' : True,
    'set_type' : False,
    'set_uri' : False,
    'set_search_uri' : False,
    'scryfall_set_uri' : False,
    'rulings_uri' : False,
    'prints_search_uri' : False,
    'collector_number' : True,
    'digital' : False,
    'rarity' : True,
    'card_back_id' : False,
    'artist' : True,
    'artist_ids' : False,
    'illustration_id' : False,
    'border_color' : False,
    'frame' : False,
    'security_stamp' : False,
    'full_art' : False,
    'textless' : False,
    'booster' : False,
    'story_spotlight' : False,
    'edhrec_rank' : False,
    'penny_rank' : False,
    'prices' : False,
    'related_uris' : False,
    'purchase_uris' : False
}

In [39]:
fullJSON = json.loads(resp.content)

selectKeys = [i for i in offlineKeys if offlineKeys[i]]
offlineJSON = { i : fullJSON[i] for i in selectKeys}

In [40]:
offlineJSON

{'object': 'card',
 'id': '6af12a34-52a1-4940-a8d5-9e9811fcd59f',
 'oracle_id': 'e245d5c9-ae3a-4246-bd74-c6fed76b9fad',
 'name': 'Temmet, Vizier of Naktamun',
 'lang': 'en',
 'released_at': '2017-04-28',
 'layout': 'normal',
 'mana_cost': '{W}{U}',
 'cmc': 2.0,
 'type_line': 'Legendary Creature — Human Cleric',
 'oracle_text': "At the beginning of combat on your turn, target creature token you control gets +1/+1 until end of turn and can't be blocked this turn.\nEmbalm {3}{W}{U} ({3}{W}{U}, Exile this card from your graveyard: Create a token that's a copy of it, except it's a white Zombie Human Cleric with no mana cost. Embalm only as a sorcery.)",
 'power': '2',
 'toughness': '2',
 'colors': ['U', 'W'],
 'color_identity': ['U', 'W'],
 'keywords': ['Embalm'],
 'legalities': {'standard': 'not_legal',
  'future': 'not_legal',
  'historic': 'legal',
  'timeless': 'legal',
  'gladiator': 'legal',
  'pioneer': 'legal',
  'explorer': 'legal',
  'modern': 'legal',
  'legacy': 'legal',
  'paup

In [41]:
len(resp.content), len(json.dumps(offlineJSON))

(5151, 1366)

In [42]:
numberCardsEDH = 15103

print('Full JSON size: {:5.1f} mb'.format(numberCardsEDH * len(resp.content) / 1000 / 1000))
print('Offl JSON size: {:5.1f} mb'.format(numberCardsEDH * len(json.dumps(offlineJSON)) / 1000 / 1000))
print('Imgs JSON size: {:5.1f} mb'.format(numberCardsEDH * len(requests.get(json.loads(resp.text)['image_uris']['small']).content) / 1000 / 1000))

Full JSON size:  77.8 mb
Offl JSON size:  20.6 mb
Imgs JSON size: 221.6 mb
